In [1]:
#from google.colab import files
import pandas as pd
import numpy as np
import calendar
# import warnings

import matplotlib.pyplot as plt

### Cargando datos de precipitación

In [143]:
# Cargamos el archivo Excel - RFF Procesado
# --------------

ruta_path = 'C:/cesar_quezada/ProyectosIA/Proyectos_PALMAS'               # -> Casa
# ruta_path = 'C:/Users/usuario/Documents/DOCUMENTOS_CONSULTOR_5_BI'      # -> Trabajo

path_calidad_precipitacion_pw = '/PROYECTO_001_analitica_descriptiva_TEA/data/external/bbdd_precipitacion/PRECIPITACION PW.xlsx'
path_calidad_precipitacion_sh = '/PROYECTO_001_analitica_descriptiva_TEA/data/external/bbdd_precipitacion/Plantilla_AGRI_Precipitacion_SH.xlsx'
path_calidad_precipitacion_tl = '/PROYECTO_001_analitica_descriptiva_TEA/data/external/bbdd_precipitacion/Precipitacion diaria y quincenal estacion TLM.xlsx'


# Cargar la hoja 2 (indexada desde 0, por lo tanto la hoja 2 es 'sheet_name=1')
df_calidad_precipitacion_pw = pd.read_excel(ruta_path + path_calidad_precipitacion_pw)
df_calidad_precipitacion_sh = pd.read_excel(ruta_path + path_calidad_precipitacion_sh,sheet_name='sheet1')
df_calidad_precipitacion_tl = pd.read_excel(ruta_path + path_calidad_precipitacion_tl,skiprows=3 ,sheet_name='sheet1')

c:\Users\CESAR QUEZADA\AppData\Local\Programs\Python\Python311\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


#### Pluviometros Palmawasi y Nuevo Horizonte

In [146]:
# Pluviometros Palmawasi y Nuevo Horizonte
# ---------------------
# ---------------------

print(f"{len(df_calidad_precipitacion_pw['NOME'].unique())} pluviometros reportados. Hay uno nuevo: 'Alto Huallaga' ?")
df_calidad_precipitacion_pw['NOME'].unique()

# Creando variable PLANTACION y tipo EQUIPO
# -----
df_calidad_precipitacion_pw['PLANTACION'] = np.where(df_calidad_precipitacion_pw['NOME'] == 'Primavera', 'Nuevo Horizonte', 'Palmawasi')
df_calidad_precipitacion_pw['EQUIPO'] = np.where(df_calidad_precipitacion_pw['NOME'] == 'Palmawasi', 'Estacion', 'Pluviometro')

# Transformando variable FECHA
# -----
# Creando variable FECHA
df_calidad_precipitacion_pw['FECHA'] = pd.to_datetime(df_calidad_precipitacion_pw["FECHA"])
df_calidad_precipitacion_pw['HORA'] = df_calidad_precipitacion_pw["FECHA"].dt.hour
df_calidad_precipitacion_pw['FECHA'] = df_calidad_precipitacion_pw['FECHA'].dt.strftime('%Y-%m-%d')

# Crear un diccionario de mapeo
mapping = {
    'Primavera': 'SE05',
    'Vivero': 'SE02',
    'Jorge Chavez': 'SE03',
    'San Cristobal': 'SE04',
    'Fray Martin': 'SE4F',
    'Ollates': 'Desconocido',
    'Yonte': 'SE04',
    'Canchis': 'Desconocido',
    'Palmawasi': 'Desconocido',
    'Guantanamo': 'SE4F',
    'Alto Huallaga': 'Desconocido'
}
# Usar map y fillna
df_calidad_precipitacion_pw['SECTOR'] = df_calidad_precipitacion_pw['NOME'].map(mapping).fillna('Desconocido')


df_calidad_precipitacion_pw.head(10)

12 pluviometros reportados. Hay uno nuevo: 'Alto Huallaga' ?


,FECHA,PRECIPITACAO,NOME,vch_Centro,PK_Tiempo,Hora,Minuto,PLANTACION,EQUIPO,HORA,SECTOR
0,2023-02-12,8.7,Primavera,1523102,20230211,7,0,Nuevo Horizonte,Pluviometro,0,SE05
1,2021-01-13,1.8,Vivero,1523102,20210113,19,0,Palmawasi,Pluviometro,0,SE02
2,2019-08-19,0.0,Jorge Chavez,1523102,20190818,7,0,Palmawasi,Pluviometro,0,SE03
3,2020-02-02,0.0,San Cristobal,1523102,20200202,19,0,Palmawasi,Pluviometro,0,SE04
4,2021-09-25,0.0,Vivero,1523102,20210925,19,0,Palmawasi,Pluviometro,0,SE02
5,2022-03-26,0.0,Fray Martin,1523102,20220325,7,0,Palmawasi,Pluviometro,0,SE4F
6,2023-01-04,0.0,Ollates,1523102,20230103,7,0,Palmawasi,Pluviometro,0,Desconocido
7,2021-03-17,0.3,Jorge Chavez,1523102,20210316,7,0,Palmawasi,Pluviometro,0,SE03
8,2022-11-06,0.0,Ollates,1523102,20221106,19,0,Palmawasi,Pluviometro,0,Desconocido
9,2023-01-25,0.0,Tomas,1523102,20230125,19,0,Palmawasi,Pluviometro,0,Desconocido


In [147]:
# Pivotar el DataFrame
df_pivot_calidad_precipitacion_pw_nh = df_calidad_precipitacion_pw.pivot(index=['FECHA', 'NOME', 'PLANTACION','EQUIPO','SECTOR'], 
                                                                         columns='Hora', values='PRECIPITACAO').reset_index()

df_pivot_calidad_precipitacion_pw_nh['PRECIPITACION_TOTAL'] = df_pivot_calidad_precipitacion_pw_nh[[0,1,5,6,7,8,10,17,19,20]].sum(axis=1,skipna=True)
df_pivot_calidad_precipitacion_pw_nh.tail(10)


Hora,FECHA,NOME,PLANTACION,EQUIPO,SECTOR,0,1,5,6,7,8,10,17,19,20,PRECIPITACION_TOTAL
20620,2024-03-16,Tomas,Palmawasi,Pluviometro,Desconocido,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,NaN,0.00
20621,2024-03-17,Alto Huallaga,Palmawasi,Pluviometro,Desconocido,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,1.03,NaN,1.03
20622,2024-03-17,Canchis,Palmawasi,Pluviometro,Desconocido,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.00,NaN,0.00
20623,2024-03-17,Fray Martin,Palmawasi,Pluviometro,SE4F,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.00,NaN,0.00
20624,2024-03-17,Guantanamo,Palmawasi,Pluviometro,SE4F,NaN,NaN,NaN,NaN,5.8,NaN,NaN,NaN,8.20,NaN,14.00
20625,2024-03-17,Jorge Chavez,Palmawasi,Pluviometro,SE03,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.00,NaN,0.00
20626,2024-03-17,Ollates,Palmawasi,Pluviometro,Desconocido,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.00,NaN,0.00
20627,2024-03-17,Primavera,Nuevo Horizonte,Pluviometro,SE05,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.00,NaN,0.00
20628,2024-03-17,San Cristobal,Palmawasi,Pluviometro,SE04,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.00,NaN,0.00
20629,2024-03-17,Tomas,Palmawasi,Pluviometro,Desconocido,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.00,NaN,0.00


In [148]:
# Agrupar por 'SEMANA' y calcular la suma de 'VALOR1' y 'VALOR2'
df_pivot_calidad_precipitacion_pw_nh_group= df_pivot_calidad_precipitacion_pw_nh.groupby(['FECHA','PLANTACION']).agg({'PRECIPITACION_TOTAL': 'sum'}).reset_index()
df_pivot_calidad_precipitacion_pw_nh_group.tail()

Hora,FECHA,PLANTACION,PRECIPITACION_TOTAL
3768,2024-03-15,Palmawasi,33.56
3769,2024-03-16,Nuevo Horizonte,0.00
3770,2024-03-16,Palmawasi,0.00
3771,2024-03-17,Nuevo Horizonte,0.00
3772,2024-03-17,Palmawasi,15.03


#### Pluviometros Shanuzi

In [149]:
# Pluviometros Shanuzi
# ---------------------
# ---------------------

df_calidad_precipitacion_sh['PLANTACION'] = 'Shanuzi'
df_calidad_precipitacion_sh.head()


,FECHA,HORA_ORIGIN,HORA,SHANUSI,CORONEL_CACAO,PALMITO,YANAYACU,USHPAYACU,KATARI,YURACQUEBRADA,PISCIGRANJA,MULERIA 01-PDO,MULERIA 07,MULERIA 05,CACAO-PINEDO,PLANTACION
0,2006-08-01,07:00:00,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Shanuzi
1,2006-08-01,19:00:00,19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Shanuzi
2,2006-08-02,07:00:00,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Shanuzi
3,2006-08-02,19:00:00,19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Shanuzi
4,2006-08-03,07:00:00,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Shanuzi


In [150]:
df_calidad_precipitacion_sh['PRECIPITACION_TOTAL'] = df_calidad_precipitacion_sh[['SHANUSI', 'CORONEL_CACAO', 'PALMITO', 'YANAYACU',
                                                                                  'USHPAYACU', 'KATARI', 'YURACQUEBRADA', 'PISCIGRANJA',
                                                                                  'MULERIA 01-PDO','MULERIA 07', 'MULERIA 05', 
                                                                                  'CACAO-PINEDO']].sum(axis=1, skipna=True)

df_calidad_precipitacion_sh.tail()

,FECHA,HORA_ORIGIN,HORA,SHANUSI,CORONEL_CACAO,PALMITO,YANAYACU,USHPAYACU,KATARI,YURACQUEBRADA,PISCIGRANJA,MULERIA 01-PDO,MULERIA 07,MULERIA 05,CACAO-PINEDO,PLANTACION,PRECIPITACION_TOTAL
12911,2024-04-03,1910-12-27 19:00:00,19,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,Shanuzi,0.00
12912,2024-04-04,1910-12-28 07:00:00,7,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,Shanuzi,0.00
12913,2024-04-04,1910-12-28 19:00:00,19,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,Shanuzi,0.00
12914,2024-04-05,1910-12-29 07:00:00,7,22.0,5.03,7.9,9.6,9.9,19.5,12.4,16.4,9.3,15.9,NaN,11.5,Shanuzi,139.43
12915,2024-04-05,1910-12-29 19:00:00,19,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,Shanuzi,0.00


In [153]:
duplicados_parciales = df_calidad_precipitacion_sh.duplicated(subset=['FECHA', 'PLANTACION', 'HORA'], keep=False)
df_duplicados_parciales = df_calidad_precipitacion_sh[duplicados_parciales]

print("Duplicados basados en FECHA, PLANTACION, HORA:")
print(df_duplicados_parciales.shape)
print(df_duplicados_parciales)

Duplicados basados en FECHA, PLANTACION, HORA:
(0, 17)
Empty DataFrame
Columns: [FECHA, HORA_ORIGIN, HORA, SHANUSI, CORONEL_CACAO, PALMITO, YANAYACU, USHPAYACU, KATARI, YURACQUEBRADA, PISCIGRANJA, MULERIA 01-PDO, MULERIA 07, MULERIA 05, CACAO-PINEDO, PLANTACION, PRECIPITACION_TOTAL]
Index: []


In [152]:
# Eliminamos duplicados: 2017-03-13
# ----
df_calidad_precipitacion_sh = df_calidad_precipitacion_sh.drop([7754,7755])

In [154]:
# Pivotar el DataFrame
df_pivot_calidad_precipitacion_sh = df_calidad_precipitacion_sh.pivot(index=['FECHA','PLANTACION'],
                                                                      columns='HORA', values='PRECIPITACION_TOTAL').reset_index()
df_pivot_calidad_precipitacion_sh.tail()

df_pivot_calidad_precipitacion_sh['PRECIPITACION_TOTAL'] = df_pivot_calidad_precipitacion_sh[[7,19]].sum(axis=1,skipna=True)
df_pivot_calidad_precipitacion_sh.head(10)

HORA,FECHA,PLANTACION,7,19,PRECIPITACION_TOTAL
0,2006-08-01,Shanuzi,0.0,0.0,0.0
1,2006-08-02,Shanuzi,0.0,0.0,0.0
2,2006-08-03,Shanuzi,0.0,0.0,0.0
3,2006-08-04,Shanuzi,0.0,0.0,0.0
4,2006-08-05,Shanuzi,0.0,0.0,0.0
5,2006-08-06,Shanuzi,0.0,0.0,0.0
6,2006-08-07,Shanuzi,0.0,0.0,0.0
7,2006-08-08,Shanuzi,0.0,0.0,0.0
8,2006-08-09,Shanuzi,0.0,0.0,0.0
9,2006-08-10,Shanuzi,0.0,0.0,0.0


#### Pluviometros Tulumayo

In [155]:
df_calidad_precipitacion_tl.head()

,Unnamed: 0,Año,Mes,Quincena,Dia,Fecha,7a,7p,24 HORAS,DIAS CON LLUVIA
0,NaN,2022,1,1q,1,2022-01-01,0.0,0.0,0.0,dia sin lluvia
1,NaN,2022,1,1q,2,2022-01-02,0.0,0.0,0.0,dia sin lluvia
2,NaN,2022,1,1q,3,2022-01-03,0.0,0.0,0.0,dia sin lluvia
3,NaN,2022,1,1q,4,2022-01-04,0.0,0.0,0.0,dia sin lluvia
4,NaN,2022,1,1q,5,2022-01-05,0.0,0.0,0.0,dia sin lluvia


In [161]:
df_calidad_precipitacion_tl['PLANTACION'] = 'Tulumayo'
df_calidad_precipitacion_tl = df_calidad_precipitacion_tl.rename(columns= {'Fecha': 'FECHA'})
df_calidad_precipitacion_tl['PRECIPITACION_TOTAL'] = df_calidad_precipitacion_tl[["7a","7p"]].sum(axis=1,skipna=True)

# ----
df_calidad_precipitacion_tl.head()

,Unnamed: 0,Año,Mes,Quincena,Dia,FECHA,7a,7p,24 HORAS,DIAS CON LLUVIA,PLANTACION,PRECIPITACION_TOTAL
0,NaN,2022,1,1q,1,2022-01-01,0.0,0.0,0.0,dia sin lluvia,Tulumayo,0.0
1,NaN,2022,1,1q,2,2022-01-02,0.0,0.0,0.0,dia sin lluvia,Tulumayo,0.0
2,NaN,2022,1,1q,3,2022-01-03,0.0,0.0,0.0,dia sin lluvia,Tulumayo,0.0
3,NaN,2022,1,1q,4,2022-01-04,0.0,0.0,0.0,dia sin lluvia,Tulumayo,0.0
4,NaN,2022,1,1q,5,2022-01-05,0.0,0.0,0.0,dia sin lluvia,Tulumayo,0.0


#### Generando base total - Presipitacion

In [164]:
# df_pivot_calidad_precipitacion_pw_nh_group.head()
# df_pivot_calidad_precipitacion_sh.head()
df_calidad_precipitacion_tl.head()

,Unnamed: 0,Año,Mes,Quincena,Dia,FECHA,7a,7p,24 HORAS,DIAS CON LLUVIA,PLANTACION,PRECIPITACION_TOTAL
0,NaN,2022,1,1q,1,2022-01-01,0.0,0.0,0.0,dia sin lluvia,Tulumayo,0.0
1,NaN,2022,1,1q,2,2022-01-02,0.0,0.0,0.0,dia sin lluvia,Tulumayo,0.0
2,NaN,2022,1,1q,3,2022-01-03,0.0,0.0,0.0,dia sin lluvia,Tulumayo,0.0
3,NaN,2022,1,1q,4,2022-01-04,0.0,0.0,0.0,dia sin lluvia,Tulumayo,0.0
4,NaN,2022,1,1q,5,2022-01-05,0.0,0.0,0.0,dia sin lluvia,Tulumayo,0.0


In [170]:
df_calidad_precipitacion_full = pd.concat([df_pivot_calidad_precipitacion_pw_nh_group[['FECHA','PLANTACION','PRECIPITACION_TOTAL']],
                                          df_pivot_calidad_precipitacion_sh[['FECHA','PLANTACION','PRECIPITACION_TOTAL']],
                                          df_calidad_precipitacion_tl[['FECHA','PLANTACION','PRECIPITACION_TOTAL']]],
                                          axis=0,
                                          ignore_index=True)


# Especifica la ruta donde deseas guardar el CSV
ruta_calidad_precipitacion_full =  '/PROYECTO_001_analitica_descriptiva_TEA/data/processed/df_calidad_precipitacion_full.csv'  # Cambia esta ruta a la que necesites
# Guardar el DataFrame en formato CSV
df_calidad_precipitacion_full.to_csv(ruta_path + ruta_calidad_precipitacion_full, index=False)
print(f'Datos guardados en {ruta_path + ruta_calidad_precipitacion_full}')

df_calidad_precipitacion_full.head()

Datos guardados en C:/cesar_quezada/ProyectosIA/Proyectos_PALMAS/PROYECTO_001_analitica_descriptiva_TEA/data/processed/df_calidad_precipitacion_full.csv


,FECHA,PLANTACION,PRECIPITACION_TOTAL
0,2019-01-01,Nuevo Horizonte,0.0
1,2019-01-01,Palmawasi,0.0
2,2019-01-02,Nuevo Horizonte,0.0
3,2019-01-02,Palmawasi,0.0
4,2019-01-03,Nuevo Horizonte,0.0
